In [14]:
import pandas as pd
import numpy as np

In [6]:
nls97 = pd.read_csv("data/nls97.csv")
nls97.set_index("personid", inplace=True)

## loc and iloc

In [10]:
gpaoverall = nls97.gpaoverall
# gpaoverall.mean()
gpaoverall.describe()

count    6004.000000
mean      281.840773
std        61.635667
min        10.000000
25%       243.000000
50%       286.000000
75%       326.000000
max       417.000000
Name: gpaoverall, dtype: float64

In [17]:
gpaoverall.quantile(np.arange(0.1,1.1,0.1))

0.1    202.0
0.2    231.0
0.3    252.0
0.4    270.0
0.5    286.0
0.6    301.0
0.7    317.0
0.8    336.0
0.9    360.0
1.0    417.0
Name: gpaoverall, dtype: float64

In [19]:
(gpaoverall>4).any()

True

## Categorical

In [21]:
landtemps = pd.read_csv("data/landtemps2019avgs.csv")

In [22]:
landtemps.elevation.quantile(np.arange(0.2,1.1,0.2))

0.2      48.0
0.4     190.5
0.6     393.2
0.8    1066.8
1.0    9999.0
Name: elevation, dtype: float64

In [30]:
# where method to create a categorical series
landtemps["elevation_group"] = np.where(landtemps.elevation>landtemps.elevation.quantile(0.8), "High","Low")
landtemps.elevation_group = landtemps.elevation_group.astype("category")
landtemps.groupby(["elevation_group"])["elevation"].agg(["count","min","max"])

,count,min,max
elevation_group,,,
High,2409,1067.0,9999.0
Low,9686,-350.0,1066.8


In [31]:
# where method to create a categorical series more than 2
landtemps["elevation_group"] = np.where(
    landtemps.elevation>landtemps.elevation.quantile(0.8), "High", 
    np.where(landtemps.elevation>landtemps.elevation.median(), "Medium","Low")
    )
landtemps.groupby(["elevation_group"])["elevation"].agg(["count", "min", "max"])

,count,min,max
elevation_group,,,
High,2409,1067.0,9999.0
Low,6056,-350.0,271.3
Medium,3630,271.4,1066.8


In [33]:
test = [
    (nls97.gpaoverall<2)&(nls97.highestdegree=="0. None"), 
    nls97.highestdegree == "0. None", 
    nls97.gpaoverall<2
    ]
result = [
    "1. Low GPA and No Diploma", 
    "2. No Diploma", 
    "3. Low GPA"
    ]

In [36]:
nls97["hsachieve"] = np.select(test, result, "4. Did Okay")

nls97[["hsachieve", "gpaoverall", "highestdegree"]].head()

,hsachieve,gpaoverall,highestdegree
personid,,,
100061,4. Did Okay,306.0,2. High School
100139,4. Did Okay,NaN,2. High School
100284,2. No Diploma,NaN,0. None
100292,4. Did Okay,345.0,4. Bachelors
100583,4. Did Okay,291.0,2. High School


In [ ]:
#use apply, more on this later, located on 277 of 439

## String cleaning

In [38]:
nls97.maritalstatus.value_counts()

Married          3066
Never-married    2766
Divorced          663
Separated         154
Widowed            23
Name: maritalstatus, dtype: int64

In [49]:
test = [
    nls97.maritalstatus == "Married", 
    nls97.maritalstatus == "Never-married", 
    nls97.maritalstatus == "Divorced",
    nls97.maritalstatus == "Separated",
    nls97.maritalstatus == "Widowed"
    ]
result = [
    "Yes", 
    "No", 
    "Yes",
    "Yes",
    "Yes"
    ]

nls97["evermarried"] = np.select(test, result, "Unknown")

nls97[["maritalstatus", "evermarried"]].head()

,maritalstatus,evermarried
personid,,
100061,Married,Yes
100139,Married,Yes
100284,Never-married,No
100292,NaN,Unknown
100583,Married,Yes


In [53]:
pd.crosstab(nls97.maritalstatus, nls97.evermarried)

evermarried,No,Yes
maritalstatus,,
Divorced,0,663
Married,0,3066
Never-married,2766,0
Separated,0,154
Widowed,0,23


In [51]:
nls97.evermarried.value_counts()

Yes        3906
No         2766
Unknown    2312
Name: evermarried, dtype: int64